In [478]:
from scipy.io import loadmat
from scipy.io import savemat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MultipleLocator
import statistics
import csv
plt.rcParams['figure.dpi'] = 900
from matplotlib import font_manager
plt.style.use('_mpl-gallery')
import matplotlib as mpl
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import TheilSenRegressor
from matplotlib import font_manager
mpl.rcParams['font.family'] = 'Times New Roman'
import scipy.io as sio
import os

In [479]:
ul_table_file = '../IMC2025/BS_congestion/ngscope_verify/0827_3/ul_whole_table.txt'
offset_value_dci = 38000+1151-30951+231
offset_value_ul = 8204+231
tx_end = 426414
rnti_ue = 15360
MAT_FILE = '../IMC2025/BS_congestion/ngscope_verify/0827_3/ul_load_cache.mat'
if os.path.exists(MAT_FILE):
    data = sio.loadmat(MAT_FILE)
    ul_table = data["ul_table"].tolist()
    load_table = data["load_table"].tolist()

pkt_size_array = [40,40,40,40,40,40,100,200,
                  200,200,300,600,630,600,700,800,900,1000,
                  900,900,970,960,1030,1000,1060,1120,1200,1250,
                  1240,1200,1140,1230,1130]
interpkt_delay_array = [4000,3000,2000,1700,1300,1000,1400,2000,
                        1500,800,800,1200,1000,800,800,800,800,800,
                        650,600,600,550,550,500,500,500,500,500,
                        450,400,350,350,300]
rate_labels = [
    f"{(size * 8 * 1000) // delay}"
    for size, delay in zip(pkt_size_array, interpkt_delay_array)
]

index_list = list(range(64))
lower_bounds = [ 0,  0, 10, 12, 14, 17, 19, 22, 26, 31, 36, 42, 49, 57, 67, 78,
    91,107,125,146,171,200,234,274,321,376,440,515,603,706,826,967,
   1132,1326,1552,1817,2127,2490,2915,3413,3995,4677,5476,6411,7505,
   8787,10287,12043,14099,16507,19325,22624,26487,31009,36304,42502,
   49759,58255,68201,79846,93479,109439,128125, 150000]
upper_bounds = [ 0, 10, 12, 14, 17, 19, 22, 26, 31, 36, 42, 49, 57, 67, 78, 91,
   107,125,146,171,200,234,274,321,376,440,515,603,706,826,967,1132,
   1326,1552,1817,2127,2490,2915,3413,3995,4677,5476,6411,7505,8787,
  10287,12043,14099,16507,19325,22624,26487,31009,36304,42502,49759,
  58255,68201,79846,93479,109439,128125,150000, 150000]

periods = [[0,9981],[13000,23010],[26000,36027],[39000,49026],[51927,62047],[65010,75036],[78010,88034],[90967,101008],[103930,114014],[117010,127040],
           [130007,140000],[143010,153012],[156010,166039],[169000,179014],[182010,192031],[195010,205026],[208000,218030],[221010,231032],
           [234000,244020],[247000,257040],[260010,270031],[273020,283079],[286000,296305],[298970,309259],[312020,322225],[325020,335237],
           [338020,348273],[351090,361330],[364090,374355],[377080,387429],[390090,400915],[403080,413372],[416130,426414]]
part_list = [ elem for elem in range(33)]

In [480]:
# pkt_file = '../IMC2025/BS_congestion/ngscope_verify/0827_3/pkt_log'
# pkt_table = []
# with open(pkt_file, 'r') as file:
#     reader = csv.reader(file)
#     next(reader)
#     for row in reader:
#         row = row[0].split('\t')
#         pkt_table.append(row)
# pkt_space = []
# for i in range(len(pkt_size_array)):
#     pkt_size = pkt_size_array[i]
#     interpkt_delay = interpkt_delay_array[i]
#     delays = []
#     for j in range(len(pkt_table)):
#         if int(pkt_table[j][4]) == pkt_size and int(pkt_table[j][5]) == interpkt_delay and int(pkt_table[j][0]) != 0:
#             delay = (int(pkt_table[j][2])-int(pkt_table[j][1]))/1000
#             delays.append(delay)
#     pkt_space.append(delays)

# part_start = 0
# part_end = 33
# fig, ax1 = plt.subplots(figsize=(12, 6))
# for i in range(part_start, part_end):
#     delays = pkt_space[i]
#     v1 = ax1.violinplot(
#             dataset=delays,
#             positions=[i - part_start],
#             widths=0.35,
#             showmeans=False,
#             showextrema=False,
#             showmedians=False
#         )
#     for part in v1['bodies']:
#         part.set_facecolor((105/255, 157/255, 203/255))  # Blue
#         part.set_edgecolor('black')
#         part.set_linewidth(0.5)
#         part.set_alpha(1)
# rate_labels = [
#     f"{(size * 8 * 1000) // delay}"
#     for size, delay in zip(pkt_size_array[part_start:part_end], interpkt_delay_array[part_start:part_end])
# ]
# ax1.set_xlabel('Sending rate (kbps)', fontsize=14)
# ax1.set_ylabel('Packet delay (ms)', fontsize=14, color='black')
# ax1.tick_params(axis='y', labelsize=14)
# ax1.tick_params(axis='x', labelsize=14)
# ax1.set_xlim(-1, part_end - part_start)
# ax1.set_xticks(range(part_end - part_start))
# ax1.set_xticklabels(rate_labels, rotation=45, ha='right', fontsize=14)
# ax1.set_ylim(top=150, bottom=0)
# plt.grid(True, linestyle='--', linewidth=0.5)
# fig.tight_layout()
# plt.savefig('/Users/liuyuxin/Desktop/packet_delay_violin_plot.pdf', dpi=900)
# plt.show()

In [481]:
# part_start = 0
# part_end = 33

# ratio_all = []
# for i in range(part_start, part_end):
#     file = f'../IMC2025/BS_congestion/ngscope_verify/0827_3/Points_size_{pkt_size_array[i]}_delay_{interpkt_delay_array[i]}.txt'
#     with open(file, 'r') as f:
#         ratio_list = []
#         reader = csv.reader(f, delimiter='\t')
#         next(reader)
#         for row in reader:
#             tti = int(row[0])
#             ratio_list.append(float(row[2]))
#     ratio_all.append(ratio_list)

# plt.figure(figsize=(12, 6))
# for i, ratio_list in enumerate(ratio_all):
#     if len(ratio_list) == 0:
#         continue
#     violin_parts = plt.violinplot(
#         dataset=ratio_list,
#         positions=[i],
#         widths=0.8,
#         showmeans=False,
#         showextrema=False,
#         showmedians=False
#     )
#     for part in violin_parts['bodies']:
#         part.set_facecolor((105/255, 157/255, 203/255))  # Blue shade
#         part.set_edgecolor('black')
#         part.set_linewidth(0.5)
#         part.set_alpha(1)

# plt.xlabel('Sending rate (kbps)', fontsize=14)
# plt.ylabel('Allocated / max requested resource', fontsize=14)
# plt.tick_params(axis='both', which='major', labelsize=14)
# plt.xlim(-1, part_end-part_start)
# plt.xticks(ticks=range(part_end-part_start), labels=rate_labels, rotation=45, ha='right', fontsize=14)
# plt.grid(True, linestyle='--', linewidth=0.5)
# # plt.yscale('log')
# plt.ylim(0, 10)
# plt.tight_layout()
# plt.savefig('/Users/liuyuxin/Desktop/packet_delay_violin_plot.pdf', dpi=900)
# plt.show()

In [482]:
# part_index = 31
# pkt_file = '../IMC2025/BS_congestion/ngscope_verify/0827_3/pkt_log'
# pkt_table = []
# with open(pkt_file, 'r') as file:
#     reader = csv.reader(file)
#     next(reader)
#     for row in reader:
#         row = row[0].split('\t')
#         pkt_table.append(row)
# pkt_space_delay = []
# pkt_space_time = []
# for i in range(len(pkt_size_array)):
#     pkt_size = pkt_size_array[i]
#     interpkt_delay = interpkt_delay_array[i]
#     delays = []
#     times = []
#     for j in range(len(pkt_table)):
#         if int(pkt_table[j][4]) == pkt_size and int(pkt_table[j][5]) == interpkt_delay and int(pkt_table[j][0]) != 0:
#             delay = (int(pkt_table[j][2])-int(pkt_table[j][1]))/1000
#             time = int(pkt_table[j][2])/1000
#             delays.append(delay)
#             times.append(time)
#     pkt_space_delay.append(delays)
#     pkt_space_time.append(times)

# ratio_file = f'../IMC2025/BS_congestion/ngscope_verify/0827_3/Points_size_{pkt_size_array[part_index]}_delay_{interpkt_delay_array[part_index]}.txt'
# ratio_tti = []
# ratio_list = []
# with open(ratio_file, 'r') as f:
#     reader = csv.reader(f, delimiter='\t')
#     next(reader)
#     for row in reader:
#         tti = int(row[0])
#         ratio_tti.append(tti)
#         ratio_list.append(float(row[2]))

# pkt_delay = pkt_space_delay[part_index]
# pkt_t = pkt_space_time[part_index]
# pkt_t = [elem - pkt_space_time[0][0] for elem in pkt_t]

# plt.figure(figsize=(12, 12))
# plt.subplot(2, 1, 1)
# plt.plot(pkt_t, pkt_delay, label='Packet Delay', linestyle='-', color='blue',marker='o', markersize=2)
# # plt.xlim(left=222000, right=223000)
# # plt.ylim(bottom=0)
# plt.tick_params(axis='both', which='major', labelsize=20)
# plt.grid(True, linestyle='--', linewidth=0.5)
# plt.xlabel('Time (ms)', fontsize=20)
# plt.ylabel('Delay (ms)', fontsize=20)
# plt.subplot(2, 1, 2)
# plt.plot(ratio_tti, ratio_list, label='Ratio', linestyle='-', color='orange',marker='o', markersize=2)
# # plt.xlim(left=222000, right=223000)
# plt.ylim(bottom=0, top=1)
# plt.tick_params(axis='both', which='major', labelsize=20)
# plt.grid(True, linestyle='--', linewidth=0.5)
# plt.xlabel('Time (ms)', fontsize=20)
# plt.ylabel('Ratio', fontsize=20)
# plt.tight_layout()
# plt.savefig('/Users/liuyuxin/Desktop/slope.pdf', dpi=900)
# plt.show()

In [483]:
def calculate_ratio_load(part_index, load_table, ul_table):
    bs_load = []

    requested_resources_upper = []
    requested_resources_mid = []
    requested_resources_lower = []

    allocated_resources = []
    ratio_update = []

    # period_start = periods[part_index][0] + 200
    # period_end = periods[part_index][-1] - 200
    period_start = periods[part_index][0]
    period_end = periods[part_index][-1]

    for j in range(len(load_table)):
        tti = load_table[j][0]
        if tti >= period_start and tti <= period_end: 
            # bs_load.append([tti, load_table[j][1]/75])
            bs_load.append([tti, load_table[j][1]/40])
        if tti > period_end:
            break
    
    nof_bsr = 0
    for k in range(len(ul_table)):
        tti = ul_table[k][0]
        if tti >= period_start and tti <= period_end:
            if ul_table[k][4] == 1 and ul_table[k][7] != -1:
                nof_bsr += 1
        if tti > period_end:
            break
    
    bsr_index = 0
    for m in range(len(ul_table)):
        tti = ul_table[m][0]
        if tti > period_end:
            break
        if tti >= period_start and tti <= period_end:
            if ul_table[m][4] == 1 and ul_table[m][7] != -1:
                upper_requested_bytes = 0
                lower_requested_bytes = 0
                allocated_bytes = 0
                overlap = 0

                bsr_index += 1

                if bsr_index == nof_bsr: # The last BSR
                    for a in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0]-ul_table[m][0]
                        if tti_diff > 0 and tti_diff <= 4 and ul_table[a][3] == 0:
                            overlap += ul_table[a][2]
                        if tti_diff > 4:
                            break
                    upper_value = upper_bounds[ul_table[m][7]]
                    lower_value = lower_bounds[ul_table[m][7]]

                    if overlap >= lower_value:
                            upper_requested_bytes += upper_value
                            lower_requested_bytes += lower_value
                    else:
                        upper_requested_bytes += (upper_value - overlap)
                        lower_requested_bytes += (lower_value - overlap)
                    
                    for b in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0] - ul_table[m][0]
                        if tti_diff >= 5 and ul_table[b][3] == 0:
                            allocated_bytes += ul_table[b][2]
                    continue

                else:
                    for a in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0]-ul_table[m][0]
                        if tti_diff > 0 and tti_diff <= 4 and ul_table[a][3] == 0:
                            overlap += ul_table[a][2]
                        if tti_diff > 4:
                            break
                    upper_value = upper_bounds[ul_table[m][7]]
                    lower_value = lower_bounds[ul_table[m][7]]

                    if overlap >= lower_value:
                        upper_requested_bytes += upper_value
                        lower_requested_bytes += lower_value
                    else:
                        upper_requested_bytes += (upper_value - overlap)
                        lower_requested_bytes += (lower_value - overlap)
                    
                    for c in range(m + 1, len(ul_table)):
                        if ul_table[c][4] == 1 and ul_table[c][7] != -1:
                            bsr_period = ul_table[c][0] - ul_table[m][0]
                            break

                    for b in range(m, len(ul_table)):
                        tti_diff = ul_table[b][0] - ul_table[m][0]
                        if tti_diff >= 5 and tti_diff <= (bsr_period+4) and ul_table[b][3] == 0:
                            allocated_bytes += ul_table[b][2]
                        if tti_diff > (bsr_period+4):
                            break

                requested_resources_upper.append(upper_requested_bytes)
                requested_resources_lower.append(lower_requested_bytes)
                mid_requested_bytes = (lower_requested_bytes + upper_requested_bytes)//2
                requested_resources_mid.append(mid_requested_bytes)
                allocated_resources.append(allocated_bytes)
                # ratio_update.append([tti, allocated_bytes/(upper_requested_bytes*(ul_table[m][7] ** 0.5 ))]) if upper_requested_bytes != 0 else ratio_update.append([tti, 5])
                # ratio_update.append([tti, allocated_bytes/(upper_requested_bytes*ul_table[m][7])]) if upper_requested_bytes != 0 else ratio_update.append([tti, 50])
                ratio_update.append([tti, allocated_bytes/upper_requested_bytes]) if upper_requested_bytes != 0 else ratio_update.append([tti, 5])
                # ratio_update.append([tti, allocated_bytes/lower_requested_bytes]) if lower_requested_bytes != 0 else ratio_update.append([tti, 5])
    return bs_load, ratio_update, allocated_resources, requested_resources_upper

In [484]:
# part_index = 17
# # part_index = 31
# bs_load, ratio_raw, allocated_resources, requested_resources_upper = calculate_ratio_load(part_index, load_table, ul_table)

# bs_load_avg = []
# ratio_update = []
# allocated_update = []
# requested_update = []
# time_list = []

# for i in range(len(ratio_raw)-1):
#     bsr_current = ratio_raw[i][0]
#     bsr_next = ratio_raw[i+1][0]
#     ratio_value = ratio_raw[i][1]
#     allocated_value = allocated_resources[i]
#     requested_value = requested_resources_upper[i]
#     load_list = []
#     for j in range(len(bs_load)):
#         if bs_load[j][0]-4 > bsr_current and bs_load[j][0]-4 <= bsr_next:
#             load_list.append(bs_load[j][1])
#         if bs_load[j][0]-4 > bsr_next:
#             break
#     if all(x == 0 for x in load_list):
#         continue
#     else:
#         non_zero_elements = [x for x in load_list if x != 0]
#         bs_load_value = np.mean(non_zero_elements)
#         time_list.append(bsr_current)
#         bs_load_avg.append(bs_load_value)
#         ratio_update.append(ratio_value)
#         allocated_update.append(allocated_value)
#         requested_update.append(requested_value)

In [485]:
# plt.figure(figsize=(12, 12))
# plt.subplot(2, 1, 1)
# plt.plot(time_list, allocated_update, label='Allocated', linestyle='-', color='blue',marker='o', markersize=2)
# plt.plot(time_list, requested_update, label='Requested', linestyle='-', color='green',marker='o', markersize=2)
# # plt.xlim(left=222000, right=223000)
# plt.xlim(left=404000, right=405000)
# plt.ylim(bottom=0)
# plt.tick_params(axis='both', which='major', labelsize=20)
# plt.grid(True, linestyle='--', linewidth=0.5)
# plt.legend(fontsize=20)
# plt.xlabel('Time (ms)', fontsize=20)
# plt.ylabel('Resource (byte)', fontsize=20)
# plt.subplot(2, 1, 2)
# plt.plot(time_list, ratio_update, label='Ratio', linestyle='-', color='orange',marker='o', markersize=2)
# # plt.xlim(left=222000, right=223000)
# plt.xlim(left=404000, right=405000)
# plt.ylim(bottom=0, top=5)
# plt.tick_params(axis='both', which='major', labelsize=20)
# plt.grid(True, linestyle='--', linewidth=0.5)
# plt.legend(fontsize=20)
# plt.xlabel('Time (ms)', fontsize=20)
# plt.ylabel('Ratio', fontsize=20)
# plt.tight_layout()
# plt.savefig('/Users/liuyuxin/Desktop/slope.pdf', dpi=900)
# plt.show()

In [486]:
# part_start = 16
# part_end = 24

# ratio_all = []
# for part_index in range(part_start, part_end):
#     _, ratio_update, _, _ = calculate_ratio_load(part_index, load_table, ul_table)
#     ratio_all.append(ratio_update)

# colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

# plt.figure(figsize=(10, 6))
# for i, ratio_update in enumerate(ratio_all):
#     if len(ratio_update) == 0:
#         continue
#     ratio_list = [row[1] for row in ratio_update]
#     sorted_data = np.sort(ratio_list)
#     yvals = np.arange(1, len(sorted_data)+1) / len(sorted_data)
#     plt.plot(sorted_data, yvals, label=f'{rate_labels[part_start+i]} kbps', color=colors[i])

# plt.xlabel('Ratio', fontsize=18)
# plt.ylabel('CDF', fontsize=18)
# plt.xlim(left=0, right=0.5)
# plt.ylim(bottom=0, top=0.2)
# plt.grid(True, linestyle='--', linewidth=0.5)
# plt.legend(fontsize=15)
# plt.tick_params(axis='both', which='major', labelsize=15)
# plt.tight_layout()
# plt.savefig('/Users/liuyuxin/Desktop/slope_3.pdf', dpi=900)
# plt.show()


In [ ]:
def calculate_ratio_load_detail_formatlab(part_index, load_table, ul_table):
    requested_resources_upper = []
    requested_resources_mid = []
    requested_resources_lower = []
    allocated_resources = []
    ratio_update = []

    requested_bsr_index = []
    overlap_detail = []
    allocated_detail = []

    period_start = periods[part_index][0]
    period_end = periods[part_index][-1]
    
    nof_bsr = 0
    for k in range(len(ul_table)):
        tti = ul_table[k][0]
        if tti >= period_start and tti <= period_end:
            if ul_table[k][4] == 1 and ul_table[k][7] != -1:
                nof_bsr += 1
        if tti > period_end:
            break
    
    bsr_index = 0
    for m in range(len(ul_table)):
        tti = ul_table[m][0]
        if tti > period_end:
            break
        if tti >= period_start and tti <= period_end:
            if ul_table[m][4] == 1 and ul_table[m][7] != -1:
                upper_requested_bytes = 0
                lower_requested_bytes = 0
                allocated_bytes = 0
                overlap = 0

                bsr_index += 1

                if bsr_index == nof_bsr: # The last BSR
                    for a in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0]-ul_table[m][0]
                        if tti_diff > 0 and tti_diff <= 4 and ul_table[a][3] == 0:
                            overlap += ul_table[a][2]
                        if tti_diff > 4:
                            break
                    upper_value = upper_bounds[ul_table[m][7]]
                    lower_value = lower_bounds[ul_table[m][7]]

                    if overlap >= lower_value:
                            upper_requested_bytes += upper_value
                            lower_requested_bytes += lower_value
                    else:
                        upper_requested_bytes += (upper_value - overlap)
                        lower_requested_bytes += (lower_value - overlap)
                    
                    for b in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0] - ul_table[m][0]
                        if tti_diff >= 5 and ul_table[b][3] == 0:
                            allocated_bytes += ul_table[b][2]
                    continue

                else:
                    for a in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0]-ul_table[m][0]
                        if tti_diff > 0 and tti_diff <= 4 and ul_table[a][3] == 0:
                            overlap += ul_table[a][2]
                        if tti_diff > 4:
                            break
                    upper_value = upper_bounds[ul_table[m][7]]
                    lower_value = lower_bounds[ul_table[m][7]]

                    if overlap >= lower_value:
                        upper_requested_bytes += upper_value
                        lower_requested_bytes += lower_value
                    else:
                        upper_requested_bytes += (upper_value - overlap)
                        lower_requested_bytes += (lower_value - overlap)
                    
                    for c in range(m + 1, len(ul_table)):
                        if ul_table[c][4] == 1 and ul_table[c][7] != -1:
                            bsr_period = ul_table[c][0] - ul_table[m][0]
                            break

                    for b in range(m, len(ul_table)):
                        tti_diff = ul_table[b][0] - ul_table[m][0]
                        if tti_diff >= 5 and tti_diff <= (bsr_period+4) and ul_table[b][3] == 0:
                            allocated_bytes += ul_table[b][2]
                        if tti_diff > (bsr_period+4):
                            break

                requested_resources_upper.append(upper_requested_bytes)
                requested_resources_lower.append(lower_requested_bytes)
                mid_requested_bytes = (lower_requested_bytes + upper_requested_bytes)//2
                requested_resources_mid.append(mid_requested_bytes)
                allocated_resources.append(allocated_bytes)
                # ratio_update.append([tti, allocated_bytes/(upper_requested_bytes*(ul_table[m][7] ** 0.5 ))]) if upper_requested_bytes != 0 else ratio_update.append([tti, 5])
                # ratio_update.append([tti, allocated_bytes/(upper_requested_bytes*ul_table[m][7])]) if upper_requested_bytes != 0 else ratio_update.append([tti, 50])
                ratio_update.append([tti, allocated_bytes/upper_requested_bytes]) if upper_requested_bytes != 0 else ratio_update.append([tti, 5])
                # ratio_update.append([tti, allocated_bytes/lower_requested_bytes]) if lower_requested_bytes != 0 else ratio_update.append([tti, 5])
    return ratio_update, allocated_resources, requested_resources_upper